In [95]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing
from sklearn.preprocessing import normalize # to normalize a matrix
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model, metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from scipy import linalg
from sklearn import metrics
import random
import pyswarm
from sklearn.model_selection import learning_curve # to analyze bias and variance of linear regression
import matplotlib.pyplot as plt # for plotting the curves
from sklearn.svm import SVC
from sklearn.svm import SVR

In [48]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)
# print(pd.Series.sort_values(ratings.movie_id).unique())

13


In [3]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

users:  6078
movies:  976


In [4]:
print("minimum movies rated by a user" ,min(ratings.groupby(["user_id"]).overall.count())) # minimum ratings given by any user
print("maximum movies rated by a user" ,max(ratings.groupby(["user_id"]).overall.count())) # maximum ratings given by any user
print("minimum ratings of a movie" ,min(ratings.groupby(["movie_id"]).overall.count())) # minimum ratings of a movie
print("maximum ratings of a movie" ,max(ratings.groupby(["movie_id"]).overall.count())) # maximum ratings of a movie

minimum movies rated by a user 5
maximum movies rated by a user 237
minimum ratings of a movie 1
maximum ratings of a movie 830


In [5]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [6]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [14]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
#         print(type(df_group))
        
        df_group_list = list()
        # converting the group of dataframes into list of dataframe for random shuffling
        for index, item in df_group:
            df_group_list.append(item)
        # randomly shuffling the list
        random.shuffle(df_group_list)
        no_of_datasets = int(len(df_group)/size);
#         print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for item in df_group_list:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
#                 print(temp.user_id.unique())
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
# temp = split_dataset(ratings, 1000)
# for item in temp:
#     print(len(item.user_id.unique()))

In [9]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [11]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [98]:
# remove movies with <5 ratings
# rating_dash = min_item_ratings_criteria(ratings, 5)
# remove users who have rated <10 items
# rating_dash = min_user_ratings_criteria(rating_dash, 10)
# print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,1000)
print(rating_dash.shape)

size =  10663
size =  9965
size =  10190
size =  9822
size =  9934
size =  11383
(61957, 8)


In [99]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

6
0
1
2
3
4
5


# Multi-criteria user-user collaborative filtering

In [15]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# manhattan_similarity(train, 1, 3)

In [16]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
# euclidean_similarity(train, 1, 3)

In [17]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# chebyshev_similarity(train, 1, 3)

In [18]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating

In [19]:
# print(test[0].head(26))

In [20]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

0
0
0


In [19]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error([1,2,3],[10,20,30])

18.0

In [20]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [21]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [22]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [23]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [24]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1
unpredicted_count = list()
# print(predicted1)

In [27]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)

6.0
0
9.34528605963
4.04347826087
11.25
We cannot predict
0
0
0
13.0
9.0
9.84158415842
10.0794775293
12.1232032854
11.9938650307
10.3560797529
12.2122356495
0
0
5.0
10.9189189189
10.5
10.2988854286
We cannot predict
0
10.0769230769
4.83495145631
11.2195121951
0
8.5
11.0324324324
7.90909090909
10.1956521739
1.0
9.10160275661
0
8.95625635809
11.4613402062
9.82075471698
11.8722891566
12.5613577023
11.0
10.0
11.4285714286
6.89156626506
8.95652173913
7.95652173913
9.42857142857
11.6071428571
11.2
5.1186440678
We cannot predict
0
11.132078505
9.203125
7.0487804878
9.05515451375
7.42857142857
7.0
11.887822226
6.76923076923
4.85714285714
11.0801409071
11.9465747519
8.1948376353
10.5135135135
10.0
0
5.36666666667
9.0
6.2
9.66666666667
10.4
8.2
7.0
13.0
4.875
0
12.0
12.1208791209
11.5591127155
0
4.0
7.74747474747
6.0
0
12.4412269004
11.1258709309
10.2
11.0
9.58536585366
11.6379444589
5.79560602623
8.175066313
10.4482315113
10.063469109
8.35709220283
9.52525914504
11.8188861219
10.3054791546
9.24

KeyboardInterrupt: 

In [28]:
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict(train[1], row.user_id, row.movie_id,euclidean_similarity, 20)
    print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
print(predicted[1])

test-size =  3646
0
1
2
3
4
5
6
Continuing
7
8
9


KeyboardInterrupt: 

In [29]:
# print("unpredicted = ",unpredicted_count[1])
print(len(predicted[1]))
threshold = 7
ttest = test[1].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[1], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[1]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

9
mae =  0.00686817891691
precision =  0.8571428571428571
recall =  0.75
f1-score =  0.7999999999999999


# Singular Value Decomposition (SVD)

In [30]:
# function to convert data into mxn matrix m -> number of users, n -> number of movies
# the matrix is normalized to take care of missing ratings
# after normalizations missing ratings can be safely assumed to be zero
# df dataframe should contain columns as 'user_id', 'movie_id', 'rate'
def convert_to_normalized_matrix(df, rating_column):
    no_of_movies = len(ratings.movie_id.unique())
    no_of_users = len(ratings.user_id.unique())
    matrix = np.zeros((no_of_users, no_of_movies))
    for index, row in df.iterrows():
        matrix[row.user_id-1][row.movie_id-1] = row[rating_column]
#     print("completed")
    return matrix,normalize_matrix(matrix)

# function to normalize a matrix by subtracting the average rating of the user
def normalize_matrix(matrix):
#     matrix[matrix == 0] = np.nan
#     matrix = np.ma.array(matrix, mask=np.isnan(matrix))

    # normalize the matrix
    normalized_matrix = np.zeros((len(matrix), len(matrix[0])))
    i=0
#     print(matrix)
    num_users = len(matrix)
    while i < num_users:
        baseline = np.sum(matrix[i])
        baseline /= np.count_nonzero(matrix[i])
        # subtract baseline value from all non-zero ratings
        normalized_matrix[i][:] = [x-baseline if x>0 else x for x in matrix[i]]
        i+=1
    return normalized_matrix
    
    
# g,h = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', 'overall']])
# print(h)

In [31]:
# function to apply SVD on normalized user-item matrix
def svd(ratings, rating_column):
    original,X = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', rating_column]], rating_column)
    print("matrix created, now decomposing it")
    u,s,v = np.linalg.svd(X) # Singular Value Decomposition
    return original,u,s,v

In [32]:
# function to crop the decomposed matrices to top-k latent features
def crop_matrices(u, s, v, k):
    # calculate average rating of the user according to 'matrix'
    cropped_u = u[:,:k] # crop matrix u
    cropped_s = s[:k] # crop array s
    cropped_v = v[:k,:] # crop matrix v
    return cropped_u, cropped_s, cropped_v
# crop_matrices(u,s,v,15)

In [33]:
# to predict the rating given by 'user' to 'item' based on SVD
# u, s, v are the decomposed matrices
# matrix is the parent matrix that is decomposed
def predict_svd(matrix, u, s, v, user, item):
    # calculate average rating of 'user' according to 'matrix'
    baseline = np.sum(matrix[user-1])
    baseline /= np.count_nonzero(matrix[user-1])
#     print("average -> ", baseline)
    p = u[user-1] # latent-features values of the user
    # s is the weight of latent-features
    q = np.transpose(v)[item-1]
    
    return baseline+sum(p*s*q)
    
# matrix,u,s,v = svd(ratings,'overall')
# print("completed")
# a,b,c = crop_matrices(u,s,v,15)
# print("completed")
# predict_svd(matrix,a,b,c,1,1)

In [34]:
# creating the decomposed matrices
matrix,u,s,v = svd(train[0], 'overall')

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it


In [35]:
# croppping the matrices to appropriate no. of latent-features
a,b,c = crop_matrices(u,s,v,15)

In [36]:
# testing the svd implementation
# fingers crossed
# predicting on our training dataset1
print("test-size = ", test[0].shape[0])
predicted[0] = list() # for storing the predicted ratings
i=0
for index, row in test[0].iterrows():
    predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
#     print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[0].append(predicted_rating)
# print(predicted[0])

test-size =  3433


In [37]:
# creating empty lists for storing the predicted values
predicted = list() # for storing the predicted ratings
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1

In [38]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    matrix,u,s,v = svd(train[i], 'overall')
    a,b,c = crop_matrices(u,s,v,15)
    for index, row in test[i].iterrows():
        predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
#         print(predicted_rating)
    print("predicted ",i)
#     unpredicted_count.append(cnt)

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it
predicted  0
matrix created, now decomposing it
predicted  1
matrix created, now decomposing it
predicted  2
matrix created, now decomposing it
predicted  3
matrix created, now decomposing it
predicted  4
matrix created, now decomposing it
predicted  5


In [39]:
# Evaluating the model
print(len(predicted[5]))
threshold = 7
ttest = test[5].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[5], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[5]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

3696
mae =  2.64854391853
precision =  0.7879243015920697
recall =  0.9258736321920226
f1-score =  0.8513469652710159


# Particle Swarm Optimization (PSO)
## (using the pyswarm library)

In [133]:
# defining the fitness funcion to be optimised by pso
# weights is the list of weights that are to be calculated by PSO
# 'user' is the user for which we want to predict the weights
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4 - row.overall)**2
    return ans

In [156]:
# function for predicting the label according to our weights got using pso
def predict_pso(xopt, test):
    predicted_list = list()
    w1 = xopt[0]
    w2 = xopt[1]
    w3 = xopt[2]
    w4 = xopt[3]
    for index, row in test.iterrows():
        prediction = w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4
        predicted_list.append(prediction)
    return predicted_list

In [40]:
# Anubhav's implementation of PSO
def pso(fitness_func, lbound, ubound, swarm_size=20, max_iter=40, dimensions=4, phip=0.5, phig=0.5, weight=0.017):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]
            
            
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/10)
        v_max.append((ubound[i]-lbound[i])/10)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
#                 if v[i][j] > v_max[j]:
#                     v[i][j] = v_max[j]
                
#                 if v[i][j] < v_min[j]:
#                     v[i][j] = v_min[j]


                # Kumud's code
                #checking the limit of velocity, if goes beyond then sack it
                if v[i][j] > v_max:
                    vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j])
                                 
                                 
                
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [176]:
# applying the particle-swarm-optimization
# xopt = the optimal input values
# fopt = the optimal objective value
df = list_datasets[0]
user_list = df.groupby(['user_id'])
lower_bound = [0, 0, 0, 0]
upper_bound = [255, 255, 255, 255]
mae = 0
count = 0
for index, item in user_list:
    # split item in train and test data
    train, test = train_test_split(item, test_size = 0.3, random_state = 1601)
    user = train
    # library implemenatation
#     xopt, fopt = pyswarm.pso(fitness_func, lower_bound, upper_bound)
    # Anubhav's implementation
    xopt, fopt = pso(fitness_func, lower_bound, upper_bound)
    print(xopt)
    print(fopt)
    # predict the values on the test dataset
    predicted_list = predict_pso(xopt, test)
#     print("test shape ", test.shape[0])
    # calculate the mean average error
#     print("checking --> ",test.overall.shape[1])
#     print("debug = ",len(predicted_list))
#     print("debug = ", len(test.overall))
    error = metrics.mean_absolute_error(test.overall.tolist(), predicted_list)
    accuracy = metrics.r2_score(test.overall.tolist(), predicted_list)
    print("mae = ",error)
    mae += error
    count += 1
    if(count==20):
        break

[0, 0, 0, 0]
264
mae =  5.5
[0, 0, 0, 0]
306
mae =  12.5
[0, 63.90047939062116, 0, 16.663029575979127]
2749424.80397
mae =  901.719768288
[0, 0, 42.18954725523602, 0.6950620000336839]
459901.78912
mae =  148.181318394
[0, 0, 0, 0]
402
mae =  13.0
[0, 0, 0, 0]
171
mae =  12.5
[0, 0, 19.409921877282674, 0]
353437.001777
mae =  146.612708352
[0, 0, 0, 0]
492
mae =  12.0
[0, 0, 0, 0]
393
mae =  10.0
[0, 0, 0, 0]
1161
mae =  9.8
[0, 0, 0, 0]
603
mae =  8.5
[0, 0, 0, 0]
492
mae =  6.5
[0, 0, 0, 0]
351
mae =  5.66666666667
[0, 0, 0, 0]
371
mae =  5.5
[0, 0, 0, 0]
486
mae =  11.0
[0, 0, 0, 0]
460
mae =  11.0
[0, 0, 0, 0]
557
mae =  12.0


KeyboardInterrupt: 

In [1]:
print("mae = ",mae/count)

NameError: name 'mae' is not defined

# Linear Regression
## (using library provided by sklearn)

In [73]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [100]:
# applying linear regression on list_datasets
# plotting style
# plt.style.use('seaborn-whitegrid')
print(len(list_datasets))
for i in range(len(list_datasets)):
    df = list_datasets[i]
    df_group = df.groupby(['user_id'])
    count = 0
    avg_error = 0
#     print("mae = ",avg_error/len(list_datasets[i].user_id.unique()))
    flag = 0
    # for plotting graph
#     fig = plt.figure()
#     ax = plt.axes()
    for key,item in df_group:
        # splitting the item in train and test data
        train, test = train_test_split(item, test_size = 0.2, random_state=1212)
        train_X = train[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        train_y = train.overall
        test_X = test[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        test_y = test.overall
        # create linear regression object
        clf = linear_model.LinearRegression(normalize=True)
        # train the model using the training sets
        clf.fit(train_X, train_y)
        prediction = clf.predict(test_X)
        st=0
        # checking the accuracy
        score = clf.score(test_X, test_y)
        
#         print(score)
        
        if(score<0.80 and flag==0):
            count+=1
#             flag=1
            # plotting the learning curve
#             r,s,t = learning_curve(linear_model.LinearRegression(), item[['criterion1', 'criterion2', 'criterion3', 'criterion4']], item.overall)
#             plt.plot(r,s)
#             plt = plot_learning_curve(linear_model.LinearRegression(),"My curve" ,item[['criterion1', 'criterion2', 'criterion3', 'criterion4']], item.overall)    
#             plt.plot(r,t)
#             plt.show()
    print("total = ",len(df_group))
    print("below average = ", count)

6
total =  1000
below average =  715
total =  1000
below average =  768
total =  1000
below average =  725
total =  1000
below average =  750
total =  1000
below average =  723
total =  1078
below average =  779


In [ ]:
# function to predict weights of a particular user using Linear Regression
# it also returns the trained classifier
def predict_weights(df, user):
    user_df = df[df['user_id']==user]
    user_df_X = user_df[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
    user_df_y = user_df.overall
    # create linear regression object
    clf = linear_model.LinearRegression(normalize=True)
    # train the model using the training sets
    clf.fit(user_df_X, user_df_y)
#     prediction = clf.predict(test_X)
    # checking the accuracy
    return clf, clf.coef_
    

In [102]:
# functions for finding similarity between two users based on their preferences to individual criteria
# based on euclidean distance
# where weights is a list of individual weights of user1
def euclidean_similarity_weighted(df, user1, user2, weights):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    for index, row in s.iterrows():
        sum1 += math.sqrt(w1*((row.criterion1_x - row.criterion1_y)**2) + \
                w2*((row.criterion2_x - row.criterion2_y)**2) + \
                w3*((row.criterion3_x - row.criterion3_y)**2) + \
                w4*((row.criterion4_x - row.criterion4_y)**2)) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity

In [103]:
# function to make predictions using some evolutionary method
def predict(df, user, item, similarity = euclidean_similarity_weighted, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        
        # get the weights preidcted using linear regression
        clf, coef = predict_weights()
        sim = similarity(df, user, v, coef) # find similarity between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating 

In [ ]:
# testing the linear regression